In [1]:
import json 
import redis
import cherrypy

In [2]:
#open redis connection
print("Insert Redis Host:")
REDIS_HOST = input()
print("Insert Redis Port:")
REDIS_PORT = int(input())
REDIS_USER = "default"
print("Insert Redis Password:")
REDIS_PASSWORD = input()

redis_client = redis.Redis(host = REDIS_HOST, port = REDIS_PORT, password = REDIS_PASSWORD, username= REDIS_USER)
print("Is connected:", redis_client.ping())


Is connected: True


In [3]:
#endpoint /devices
class Devices:
    exposed = True

    def GET(self, *path, **query):
        keys = redis_client.keys("*:battery")
        keys = [k.decode().split(':')[0] for k in keys]
        output_dict = {"mac_addresses" : keys}
        return json.dumps(output_dict)



#endpoint /devices/{mac_address} 
class DeviceList:
    exposed = True

    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad request: missing MAC address.')
        
        if len(redis_client.keys(f"{path[0]}:*"))==0:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address.')

        if query.get("from") is None:
            raise cherrypy.HTTPError(400, 'Bad request: missing start time.')
        if query.get("to") is None:
            raise cherrypy.HTTPError(400, 'Bad request: missing end time.')

        
        battery_levels = redis_client.ts().range(f"{path[0]}:battery", query.get("from"), query.get("to"))
        power_plugged = redis_client.ts().range(f"{path[0]}:power", query.get("from"), query.get("to"))

        output_dict = {
            "mac_address" : path[0],
            "timestamps" : [ts for ts, bl in battery_levels],
            "battery_levels" : [bl for ts, bl in battery_levels],
            "power_plugged" : [pp for ts, pp in power_plugged]
        }

        return json.dumps(output_dict)

    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad request: missing MAC address.')

        if len(redis_client.keys(f"{path[0]}:*"))==0:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address.')
        
        redis_client.delete(f"{path[0]}:battery")
        redis_client.delete(f"{path[0]}:power")


In [4]:
if __name__== '__main__':
    conf = {
        '/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}
    }

    #create link between the class and the endpoint
    cherrypy.tree.mount(Devices(), '/devices', conf)
    cherrypy.tree.mount(DeviceList(), '/device', conf)
  
    #specify the IP address of the host and host
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080}) 

    #start and keep it online
    cherrypy.engine.start()
    cherrypy.engine.block()

[24/Jan/2023:09:26:21] ENGINE Bus STARTING
[24/Jan/2023:09:26:21] ENGINE Started monitor thread 'Autoreloader'.
[24/Jan/2023:09:26:21] ENGINE Serving on http://0.0.0.0:8080
[24/Jan/2023:09:26:21] ENGINE Bus STARTED
172.3.133.85 - - [24/Jan/2023:09:26:34] "GET /devices HTTP/1.1" 200 55 "" "python-requests/2.28.1"
172.3.150.40 - - [24/Jan/2023:09:26:37] "GET /device/0x8ecce21a8aee?from=1674505228510&to=1674505282985 HTTP/1.1" 200 1518 "" "python-requests/2.28.1"
172.3.22.214 - - [24/Jan/2023:09:26:37] "GET /device/0xbb7de14c93be?from=1674505228510&to=1674505282985 HTTP/1.1" 200 94 "" "python-requests/2.28.1"
172.3.22.214 - - [24/Jan/2023:09:27:10] "GET /device/0x8ecce21a8aee?from=1674552201601&to=1674552205640 HTTP/1.1" 200 218 "" "python-requests/2.28.1"
172.3.22.214 - - [24/Jan/2023:09:27:10] "GET /device/0xbb7de14c93be?from=1674552201601&to=1674552205640 HTTP/1.1" 200 192 "" "python-requests/2.28.1"
172.3.22.214 - - [24/Jan/2023:09:27:24] "GET /devices HTTP/1.1" 200 55 "" "python-requ

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1e2d0164-f620-4188-849f-f2bd90b2ba46' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>